In [1]:
import os, sys, cv2, pydicom
import numpy as np

In [2]:
DPATH = '/home/ubuntu/datasets/chest/CBIS-DDSM'
cases = os.listdir(DPATH)

In [3]:
import json

# with open('.cbis_cache.json', 'w') as fl:
#     json.dump(tosave, fl, indent=4)
    
with open('.cbis_cache.json') as fl:
    tosave = json.load(fl)

print(len(tosave))

3031


In [4]:
import matplotlib.pyplot as plt

In [5]:
# mass = [ent for ent in tosave if 'Mass' in ent['image']]
# print(len(mass))

In [6]:
# tosave = mass

In [7]:
def gather_rois(case):
    rois = []
    for mpath in case['masks']:
        with pydicom.dcmread(mpath) as db:
            mask = db.pixel_array[:]
        roi = np.nonzero(mask)
        rois.append(roi)
    return rois

roi_lookup = {}
for ii, case in enumerate(tosave):
    roi_lookup[case['sid']] = gather_rois(case)
    
    sys.stdout.write('%d/%d   \r' % (ii, len(tosave)))
    sys.stdout.flush()
#     break

In [8]:
from random import randint

def draw_sample(rawimg, rawmask, yy, xx, size=256, pad=64, case=None):
    size += pad
    half = size//2
    x0, y0 = max(0, xx-half), max(0, yy-half)
    xf, yf = min(rawimg.shape[1]-1, x0+size), min(rawimg.shape[0]-1, y0+size)
    x0, y0 = xf - size, yf - size
    
    img = rawimg[y0:yf, x0:xf]
    mask = rawmask[y0:yf, x0:xf]
    try:
        assert img.shape == (size, size)
        assert mask.shape == (size, size)
    except:
        raise Exception('Sample mismatch: %s %s' %(img.shape, mask.shape))
    return img, mask

SAVEPATH = '/home/ubuntu/datasets/tissue/cancers'
import h5py
__mismatch = 0
__toobig = []
def sample_boxes(case, ratio=2000, maxout=50):
    global __mismatch
    samples = []
    sid = case['sid']
    many = roi_lookup[sid]
    with pydicom.dcmread(case['image']) as db:
        raw = db.pixel_array[:]
    assert len(many) > 0
    for mii, (roiY, roiX) in enumerate(many):
        if len(roiX) > 320**2: 
            __toobig.append([case, mii])
            continue
#         print(len(roiX))
        samps = max(1, min(len(roiY) // ratio, maxout))
        assert samps > 0
        with pydicom.dcmread(case['masks'][mii]) as db:
            rawmask = db.pixel_array[:]
        try:
            assert raw.shape == rawmask.shape
        except:
            # some masks have been scaled down ~10% for some reason
            scale = raw.shape[0] / rawmask.shape[0]
            rawmask = cv2.resize(rawmask, (0,0), fx=scale, fy=scale)
            print('Resize', rawmask.shape, raw.shape)
            __mismatch += 1
            
        ym, xm = int(np.mean(roiY)), int(np.mean(roiX))
        img, mask = draw_sample(raw, rawmask, ym, xm, case=case)
#         if img is None: continue # some error sampling
            
        ref = '%s' % (sid)
        mask_path = '%s/%s.jpg'% (SAVEPATH, ref)
        cv2.imwrite(mask_path, mask)
        img_path = '%s/%s.npy' % (SAVEPATH, ref)
        np.save(img_path, img)
#         samples.append((img, mask))

skip = 0
for ii, case in enumerate(tosave[skip:]):
    sample_boxes(case)
    
    sys.stdout.write('%d/%d  Miss: %d  Toobig: %d \r' % (
        ii, len(tosave)-skip, __mismatch, len(__toobig)))
    sys.stdout.flush()

Resize (5296, 3436) (5296, 3436)
Resize (5356, 2581) (5356, 2581)
Resize (5356, 2852) (5356, 2851)
Resize (5296, 2566) (5296, 2566)
Resize (5236, 2971) (5236, 2971)
Resize (5026, 2837) (5026, 2836)
Resize (5296, 2566) (5296, 2566)
Resize (5281, 2477) (5281, 2476)
Resize (5266, 2762) (5266, 2761)
Resize (5356, 2326) (5356, 2326)
Resize (5386, 3437) (5386, 3436)
Resize (5386, 3316) (5386, 3316)
Resize (5386, 3467) (5386, 3466)
Resize (5296, 3062) (5296, 3061)
Resize (5386, 3437) (5386, 3436)
Resize (5386, 3722) (5386, 3721)
Resize (5386, 3286) (5386, 3286)
Resize (5236, 3316) (5236, 3316)
Resize (5011, 2596) (5011, 2596)
Resize (5221, 2971) (5221, 2971)
Resize (5341, 3496) (5341, 3496)
Resize (5236, 2716) (5236, 2716)
Resize (5386, 3092) (5386, 3091)
Resize (5386, 3152) (5386, 3151) 
Resize (5341, 3737) (5341, 3736) 
Resize (5326, 3422) (5326, 3421) 
Resize (5386, 3601) (5386, 3601) 
Resize (5386, 3241) (5386, 3241) 
Resize (5311, 3196) (5311, 3196) 
Resize (5386, 3692) (5386, 3691) 
Res

In [ ]:
with open('.toobig.json', 'w') as fl:
    json.dump(__toobig, fl, indent=4)
